In [1]:
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.4.2,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.2_2.12:0.74.0,com.amazonaws:aws-java-sdk-bundle:1.12.26 pyspark-shell"

In [2]:
import os

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
NESSIE_URI = "http://nessie:19120/api/v1"
AWS_ACCESS_KEY = "minio"
AWS_SECRET_KEY = "minio123"
AWS_S3_ENDPOINT = "http://minio:9000"
WAREHOUSE = "s3a://lake/"

In [5]:
conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
        # .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.13.1,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
        .set("spark.sql.execution.pyarrow.enabled", "true")
        .set(
            "spark.jars.packages",
            f"org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.4.2,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.2_2.12:0.74.0",
        )
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        # .set('spark.sql.catalog.nessie.s3.endpoint', AWS_S3_ENDPOINT)
        # .set('spark.sql.catalog.nessie.warehouse', WAREHOUSE)
        .set("spark.sql.catalog.nessie.warehouse", "file://" + os.getcwd() + "/spark_warehouse/iceberg")
        # .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set('spark.hadoop.fs.s3a.access.key', 'minio')
        .set('spark.hadoop.fs.s3a.secret.key', 'minio123')
        .set('spark.hadoop.fs.s3a.endpoint', "http://minio:9000")
        .set('spark.hadoop.fs.s3a.connection.ssl.enabled', "false")
        .set('spark.hadoop.fs.s3a.path.style.access', "true")
        .set(
            "spark.sql.extensions",
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions",
        )
)

In [6]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.2_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.2_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3670911a-0a88-4e25-bd28-b5bd11075e51;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;1.4.2 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.2_2.12;0.74.0 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.2_2.12/1.4.2/iceberg-spark-runtime-3.2_2.12-1.4.2.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.2_2.12;1.4.2!iceberg-spark-runtime-3.2_2.12.jar (2861ms)
downloading https://repo1.maven.org/maven2/org/projectnessie/nessie-integrations/nessie-spark-extensions-3.2_2.12/0.74.0/nessie-spark-extensions-3.2_2.12-0.74.0.jar ...
	[SUCCESSFUL ]

In [7]:
spark.sql("CREATE TABLE nessie.names (name STRING) USING iceberg;").show()

++
||
++
++



In [8]:
spark.sql("INSERT INTO nessie.names VALUES ('Alex Merced'), ('Dipankar Mazumdar'), ('Jason Hughes')")

DataFrame[]

In [9]:
spark.sql("SELECT * FROM nessie.names;").show()

+-----------------+
|             name|
+-----------------+
|      Alex Merced|
|Dipankar Mazumdar|
|     Jason Hughes|
+-----------------+

